### 1.  country detelction  (not used in our analysis) 

In [ ]:
# predict country usuing name: 
from nationality_predictor import predict
import json

exa=predict("Xinyi")
exa1 = json.loads(exa)
country,prob,name = [],[],[]
name.append(exa1["name"])
for result in exa1["countries"]:
    country.append(result[u'country_name'])
    prob.append(result[u'probability'])
df = pd.DataFrame([name,country,prob]).T
df.rename(columns={0:"name",1:"country",2:"prob"},inplace=True)

df["name"]=df["name"].ffill()
df

### 2. gender detection

In [ ]:
import pandas as pd
from genderize import Genderize

# using detected name database (prob >0.7)
UK_genders=pd.read_csv("N:\\Bibliometrics\\datasets\\Gender\\namesdataset_UK.csv",encoding = 'latin-1')
UK_genders.rename(columns={'name':'FIRSTNAME'},  inplace=True)
UK_genders['FIRSTNAME'] = UK_genders['FIRSTNAME'].str.upper() 
UK_genders=UK_genders.drop_duplicates(["FIRSTNAME"])
UK_genders=UK_genders.drop(["count","country_id"],axis=1)
UK_genders_1=UK_genders[UK_genders["probability"]>=0.7]

In [ ]:
UK_genders_1.rename(columns={"FIRSTNAME":"FIRST_NAME"},inplace=True)
all_records_name=all_records_name.merge(UK_genders_1,on="FIRST_NAME",how="left")

In [ ]:
# import manual detection
encoding = 'utf-8'

genders_1=pd.read_csv("N:\\Bibliometric_Germany\\KB data\\processed\\id_gender_set_one.csv",encoding = encoding)
genders_2=pd.read_csv("N:\\Bibliometric_Germany\\KB data\\processed\\id_gender_set_two.csv",encoding = encoding)
genders_3=pd.read_csv("N:\\Bibliometric_Germany\\KB data\\processed\\name_dataset_manual_1996_2019_Abrar.csv",encoding = encoding)
genders_4=pd.read_csv("N:\\Bibliometric_Germany\\KB data\\processed\\name_dataset_manual_1996_2019_Chowdhury.csv",encoding = encoding)

genders=pd.concat([genders_1,genders_2,genders_3,genders_4])
genders=genders[["AUTHOR_ID","gender"]]
genders.drop_duplicates(['AUTHOR_ID'],inplace=True)

all_records_ID=all_records[["AUTHOR_ID","AUTHOR_ID_NEW"]]
genders=genders.merge(all_records_ID,on="AUTHOR_ID",how="left")
genders["AUTHOR_ID_NEW"]=genders["AUTHOR_ID_NEW"].fillna(value=genders['AUTHOR_ID'])

genders.drop(columns={"AUTHOR_ID"},inplace=True)



In [ ]:

all_records_name_0=all_records_name[all_records_name["gender"].notnull()]
all_records_name_1=all_records_name[all_records_name["gender"].isnull()]

all_records_name_1=all_records_name_1.merge(genders,on="AUTHOR_ID_NEW",how="left")
all_records_name_1.drop(columns={"gender_x","probability"},inplace=True)
all_records_name_1.rename(columns={"gender_y":"gender"},inplace=True)


In [ ]:
all_records_name_2=all_records_name_1[all_records_name_1["gender"].isnull()]
all_records_name_1=all_records_name_1[all_records_name_1["gender"].notnull()]

all_records_name_2["FIRST_NAME"]=all_records_name_2["FIRST_NAME"].str.replace(' [A-Z]*.','')

names = all_records_name_2.drop_duplicates("FIRST_NAME")
names=names["FIRST_NAME"]

In [ ]:
# using genderize.io 

#control the number of names in the range of 100000
resultDF = pd.DataFrame(Genderize(api_key='09102439c490f52765cc4cf1bd390bb5').get(names))
resultDF_1=resultDF[resultDF["probability"]>0.6]

In [ ]:
all_records_name_2.drop(columns={"gender"},inplace=True)
resultDF_1.drop(columns={"probability","count"},inplace=True)
resultDF_1.rename(columns={"name":"FIRST_NAME"},inplace=True)
all_records_name_2=all_records_name_2.merge(resultDF_1,on="FIRST_NAME",how="left")

all_records_name_3=all_records_name_2[all_records_name_2["gender"].isnull()]
all_records_name_2=all_records_name_2[all_records_name_2["gender"].notnull()]

In [ ]:
#another manual name datset 
name_dataset_1=pd.read_csv("N:\\Bibliometric_Germany\\KB data\\others\\names_dataset.csv",encoding=encoding)
name_dataset_2=pd.read_csv("N:\\Bibliometric_Germany\\KB data\\others\\universalnames_dataset.csv",encoding=encoding)
name_dataset=pd.concat([name_dataset_1,name_dataset_2])

In [ ]:
def substitue_gender(x):
    if (x=="F"): 
        x="female"
    elif (x=="M"):
        x="male"
    else:
        pass
    
    return x



name_dataset.drop(columns={"index","Unnamed: 0"},inplace=True)
name_dataset.rename(columns={"name":"FIRST_NAME","sex":"gender"},inplace=True)
name_dataset.drop_duplicates(inplace=True)
name_dataset["gender"]=name_dataset["gender"].map(lambda x: substitue_gender(x))
name_dataset["FIRST_NAME"]=name_dataset['FIRST_NAME'].map(lambda x: x.lower())

In [ ]:
all_records_name_3=all_records_name_3.merge(name_dataset,on="FIRST_NAME",how="left")
all_records_name_3.drop(columns={"gender_x"},inplace=True)
all_records_name_3.rename(columns={"gender_y":"gender"},inplace=True)

all_records_name_4=all_records_name_3[all_records_name_3["gender"].isnull()]
all_records_name_3=all_records_name_3[all_records_name_3["gender"].notnull()]

In [ ]:
all_records_name_0.drop(columns={"probability"},inplace=True)

all_records_name_0=pd.concat([all_records_name_0,all_records_name_1,all_records_name_2,all_records_name_3])

all_records_name_0.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\10_AuthorID_name_gender.csv",encoding=encoding)
all_records_name_4.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\10_AuthorID_name_gender_unknown.csv",encoding=encoding)

### 3. machine learning gender detection 

In [ ]:
encoding = 'utf-8'

# EDA packages
import pandas as pd
import numpy as np

# ML Packages
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer

# trains dataset
df_name=all_records_name_0[all_records_name_0["FIRST_NAME"].notnull()][["FIRST_NAME","gender"]]
df_name["FIRST_NAME"]=df_name['FIRST_NAME'].map(lambda x: x.lower())
df_name=df_name[df_name["gender"]!="s"]
df_name["gender"].replace({'female':0,'male':1},inplace=True)

In [ ]:
Xfeatures =df_name['FIRST_NAME']

# Feature Extraction 
cv = CountVectorizer()
X = cv.fit_transform(Xfeatures)
cv.get_feature_names()

In [ ]:
from sklearn.model_selection import train_test_split

y = df_name.gender

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

In [ ]:
# predict unknown name
def genderpredictor(a):
    test_name = [a]
    vector = cv.transform(test_name).toarray()
    if clf.predict(vector) == 0:
        b="female"
    else:
        b="male"
        
    return b

In [ ]:
unknown_names=all_records_name_4.copy()
unknown_names["gender"]=unknown_names["FIRST_NAME"].apply(lambda x:genderpredictor(x))

unknown_names.to_csv("N:\\Bibliometric_Germany\\KB data\\processed\\new_2021_April\\10_AuthorID_name_gender_predicted.csv",encoding=encoding)